In [1]:
# install required packages
# !pip install tensorflow
!pip install keras_tuner


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 18.4 MB/s eta 0:00:00


In [2]:
# import required packages
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Flatten, Convolution2D, BatchNormalization
from tensorflow.keras.layers import ReLU, MaxPool2D, AvgPool2D, GlobalAvgPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
import keras_tuner as kt
from sklearn.model_selection import train_test_split


In [3]:

# load the CIFAR-10 dataset from keras
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Normalize the image pixel values
img_train = x_train.astype('float32') / 255.0
img_test = x_test.astype('float32') / 255.0

# split the train data into train and validation sets
x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=0.5)


170498071/170498071 [==============================] - 3s 0us/step


In [4]:
y_train.shape

(25000, 1)

In [5]:
# function to build an hypermodel
# takes an argument from which to sample hyperparameters
def build_model(hp):
  inputs = Input(shape = (32, 32, 3)) #input layer
  x = inputs
  # iterate a number of conv blocks from min_value to max_value
  # tune the number of filters
  # choose an optimal value from min_value to max_value
  for i in range(hp.Int('conv_blocks',min_value = 3, max_value = 5, default=3)): # Int specifies the dtype of the values
    filters = hp.Int('filters_' + str(i),min_value = 32,max_value = 256, step=32) 
    for _ in range(2):
      # define the conv, BatchNorm and activation layers for each block
      x = Convolution2D(filters, kernel_size=(3, 3), padding= 'same')(x)
      x = BatchNormalization()(x)
      x = ReLU()(x)
    # choose an optimal pooling type
    if hp.Choice('pooling_' + str(i), ['avg', 'max']) == 'max': # hp.Choice chooses from a list of values
        x = MaxPool2D()(x)
    else:
        x = AvgPool2D()(x)
  x = GlobalAvgPool2D()(x) # apply GlobalAvG Pooling
  # Tune the number of units in the  Dense layer
  # Choose an optimal value between min_value to max_value
  x = Dense(hp.Int('Dense units',min_value = 30, max_value = 100, step=10, default=50), activation='relu')(x)
  outputs = Dense(10, activation= 'softmax')(x) # output layer 
  print(outputs)
  # define the model
  model = Model(inputs, outputs)
  # Tune the learning rate for the optimizer
  # Choose an optimal value frommin_value to max_value
  model.compile(optimizer= Adam(hp.Float('learning_rate', min_value = 1e-4, max_value =1e-2, sampling='log')), 
                loss= 'sparse_categorical_crossentropy', metrics = ['accuracy'])
  return model

In [6]:
# initialize tuner to run the model.
# using the Hyperband search algorithm
tuner = kt.Hyperband(
    hypermodel=build_model,
    objective='val_accuracy',
    max_epochs=30,
    hyperband_iterations=2,
    directory="Keras_tuner_dir",
    project_name="Keras_tuner_Demo")


KerasTensor(type_spec=TensorSpec(shape=(None, 10), dtype=tf.float32, name=None), name='dense_1/Softmax:0', description="created by layer 'dense_1'")


In [8]:
# Initialize a random search tuner
# using the Resnet architecture
# and the Random Search algorithm
tuner = kt.tuners.RandomSearch(
    kt.applications.HyperResNet(input_shape=(32, 32, 3), classes=1),
    objective='val_accuracy',
    max_trials=3)


INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json
INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json


In [9]:
y_train.shape
y_val.shape

(25000, 1)

In [11]:
# Run the search
tuner.search(x_train, y_train,
             validation_data=(x_test, y_test),
             epochs=1,
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=2)], verbose = True)


Trial 16 Complete [00h 02m 32s]
val_accuracy: 0.10000000149011612

Best val_accuracy So Far: 0.10000000149011612
Total elapsed time: 00h 05m 19s
INFO:tensorflow:Oracle triggered exit


In [12]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(1)[0]

# get the best model
best_model = tuner.get_best_models(1)[0]


In [13]:
nblocks = best_hps.get('conv_blocks')
print(f'Number of conv blocks: {nblocks}')
for hyparam in [f'filters_{i}' for i in range(nblocks)] + [f'pooling_{i}' for i in range(nblocks)] + ['Dense units'] + ['learning_rate']:
    print(f'{hyparam}: {best_hps.get(hyparam)}')


KeyError: 'conv_blocks does not exist.'

In [14]:
# display model structure
plot_model(best_model, 'best_model.png', show_shapes=True)

# show model summary
best_model.summary()


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.
Model: "ResNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 38, 38, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 16, 16, 64)   9408        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)

# Training the model

In [15]:
# Build the model with the optimal hyperparameters
# train the model.
model = tuner.hypermodel.build(best_hps)
model.fit(x_train, y_train, 
          validation_data= (x_val,y_val), 
          epochs= 25,
           callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)])

Epoch 1/25
782/782 [==============================] - 156s 144ms/step - loss: 0.0000e+00 - accuracy: 0.0971 - val_loss: 0.0000e+00 - val_accuracy: 0.1029
Epoch 2/25
782/782 [==============================] - 104s 133ms/step - loss: 0.0000e+00 - accuracy: 0.0971 - val_loss: 0.0000e+00 - val_accuracy: 0.1029
Epoch 3/25
782/782 [==============================] - 103s 132ms/step - loss: 0.0000e+00 - accuracy: 0.0971 - val_loss: 0.0000e+00 - val_accuracy: 0.1029
Epoch 4/25
782/782 [==============================] - 103s 131ms/step - loss: 0.0000e+00 - accuracy: 0.0971 - val_loss: 0.0000e+00 - val_accuracy: 0.1029
Epoch 5/25
782/782 [==============================] - 102s 131ms/step - loss: 0.0000e+00 - accuracy: 0.0971 - val_loss: 0.0000e+00 - val_accuracy: 0.1029
Epoch 6/25
782/782 [==============================] - 103s 132ms/step - loss: 0.0000e+00 - accuracy: 0.0971 - val_loss: 0.0000e+00 - val_accuracy: 0.1029


# Evaluate the result

In [16]:
# evaluate the result
eval_result = model.evaluate(x_test, y_test)
print(f"test loss: {eval_result[0]}, test accuracy: {eval_result[1]}")

313/313 [==============================] - 11s 35ms/step - loss: 0.0000e+00 - accuracy: 0.1000
test loss: 0.0, test accuracy: 0.10000000149011612
